In [1]:
from __future__ import division
import calendar
import time
import datetime
import ConfigParser
import getopt,sys
import threading
import logging
import logging.config
from logging.handlers import RotatingFileHandler
from pymongo import MongoClient
from ripe.atlas.cousteau import AtlasResultsRequest, ProbeRequest, Probe
from socket import socket
from datetime import datetime, timedelta
import csv
import os.path
import numpy
import multiprocessing
import threading

In [2]:
def totimestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    # return td.total_seconds()
    return(td.microseconds + (td.seconds + td.days * 86400))

In [3]:
def getProbeList(asn,testid):
    # print('getProbeList')
    test_id = testid
    filters = {"id": test_id, "asn": asn}
    probe_list = []
    probes = []
    try:
        print('ProbeRequest')
        probes = ProbeRequest(**filters)
    except:
        print "error getting probes", probes

    try:
        if probes:
            for probe in probes:
                probe_list.append(probe["id"])  # add the probe ID to the list
    except:
        e = sys.exc_info()
        print ("Probe error", str(e))
        probe_list = []

    # For testing get the first 10 probes
    k= 10
    probe_list2 = probe_list[0:10]
    
    return probe_list2

In [4]:
# Define a function getASNResults() to get built-in traceroute measurements for each ASN in list of source asns
# from RIPE servers
def getASNResults(asn, testid, start_time, stop_time, probe_list2):
    # print('getASNResults')
    rtt_average = 0
    rtt_mean = 0
    rtt_median = 0
    rtt_std = 0
    rtt_var = 0

    # print('Probelist', probe_list2)


    # Get the all the measurments of interest for these probes
    if len(probe_list2) > 0:
        kwargs = {
            "msm_id": testid,
            "start": start_time,
            "stop": stop_time,
            "probe_ids": probe_list2
        }
        try:
            print('atlasRequest')
            is_success, results = AtlasResultsRequest(**kwargs).create()
        except:
            e = sys.exc_info()
            print('Error AtlasResultsRequest', str(e))
            return (0, 0, 0, 0, 0)
        if is_success:
            print('atlas success')
            try:
                rtt_list = []
                for res in results:
                    rtt_list.append(res['avg'])
                    # Convert to numpy array
                arr = numpy.array(rtt_list)
                rtt_average = numpy.average(arr)
                rtt_mean = numpy.mean(arr)
                rtt_median = numpy.median(arr)
                rtt_std  = numpy.std(arr)
                rtt_var = numpy.var(arr)
            except:
                rtt_average = 0
                rtt_mean = 0
                rtt_median = 0
                rtt_std = 0
                rtt_var = 0
                e = sys.exc_info()
                print('Error AtlasRequestResults2', str(e))
        else:
            print('AtlasRequestResultsNotSuccessful')
            rtt_average = 0
            rtt_mean = 0
            rtt_median = 0
            rtt_std = 0
            rtt_var = 0
    return(rtt_average, rtt_mean, rtt_median, rtt_std, rtt_var)

In [5]:
def doDay(asn, id, start, stop, writer, probelist):
    
    try:
        rtt_average, rtt_mean, rtt_median, rtt_std, rtt_var = getASNResults(asn, id, start, stop, probelist)
        # Prep row for CSV File
        row_dict = {}
        row_dict['ASN'] = asn
        row_dict['MSM_ID'] = id
        # row_dict['Date'] = start.strftime('%m/%d/%Y,%H:%M:%S')
        row_dict['Timestamp'] = totimestamp(start)
        row_dict['RTT'] = rtt_average
        row_dict['Mean'] = rtt_mean
        row_dict['Median'] = rtt_median
        row_dict['Std'] = rtt_std
        row_dict['Var'] = rtt_var
        writer.writerow(row_dict)
        print row_dict
    except:
        e = sys.exc_info()
        print ("Error", str(e))


In [6]:
def doAsn(asn):
    # print("Starting doAsn process for ",asn)
    results_file = 'atlasdata_6_' + str(asn) +'.csv'
    if os.path.isfile(results_file):
        os.remove(results_file)
    # 1013 = E-root server, 1012488 = Google
    msm_ids = 1013, 1009
    with open(results_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for id in msm_ids:
            probelist = getProbeList(asn, id)
            for month in range(2,6):
                for day in range(1,30):
                    #print("ASN:"+str(asn)+" "+str(month) + "/" + str(day))
                    try:
                        start = datetime(2020, month, day, 0)
                        stop = datetime(2020, month, day+1, 0)
                        print("ASN:" + str(asn) + " " + str(id) + " " +str(month) + "/" + str(day) )
                        #p = multiprocessing.Process(target=doDay, args=(asn, id, start, stop, writer,))
                        #p.start()
                        #p.join(30)
                        doDay(asn, id, start, stop, writer, probelist)
                    except:
                        e = sys.exc_info()
                        print ("Error", str(e))
                        
    print('Finished:', asn)


In [ ]:

csv_columns = ['ASN', 'MSM_ID', 'Timestamp', 'RTT', 'Mean', 'Median', 'Std', 'Var']
doAsn(7922)




In [ ]:
p = []
p_id=0

csv_columns = ['ASN', 'MSM_ID', 'Timestamp', 'RTT', 'Mean', 'Median', 'Std', 'Var']
ASNs = [7922, 30036] #3269, 30036, 10796, 11351, 11426, 11427, 12271, 20001, 19108, 11492, 30722, 1267, 12874]
for i in ASNs:
    p.append(multiprocessing.Process(target=doAsn, args=(i,)))
    p[p_id].start()
    p_id = p_id+1

p_id=0
for i in ASNs:
    p[p_id].join()
    p_id = p_id+1
    
print('All Finished')
    

ProbeRequest
ProbeRequest
ASN:30036 1013 2/1
atlasRequest
atlas success
{'Std': 2.000398799240835, 'MSM_ID': 1013, 'Timestamp': 1580515200, 'Median': 18.46879583335, 'RTT': 18.546028856483613, 'Var': 4.001595356004175, 'ASN': 30036, 'Mean': 18.546028856483613}
ASN:30036 1013 2/2
atlasRequest
ASN:7922 1013 2/1
atlasRequest
atlas success
{'Std': 38.128825833316256, 'MSM_ID': 1013, 'Timestamp': 1580515200, 'Median': 15.629571666699999, 'RTT': 17.57461351449551, 'Var': 1453.807359427365, 'ASN': 7922, 'Mean': 17.57461351449551}
ASN:7922 1013 2/2
atlasRequest
atlas success
{'Std': 2.097889137571033, 'MSM_ID': 1013, 'Timestamp': 1580601600, 'Median': 18.188965, 'RTT': 18.62848754629639, 'Var': 4.401138833538534, 'ASN': 30036, 'Mean': 18.62848754629639}
ASN:30036 1013 2/3
atlasRequest
atlas success
{'Std': 218.00428319983737, 'MSM_ID': 1013, 'Timestamp': 1580601600, 'Median': 15.795459, 'RTT': 23.78789260897114, 'Var': 47525.8674934749, 'ASN': 7922, 'Mean': 23.78789260897114}
ASN:7922 1013 2/3

ASN:30036 1013 2/19
atlasRequest
atlas success
{'Std': 10.268199832144465, 'MSM_ID': 1013, 'Timestamp': 1581984000, 'Median': 15.579139, 'RTT': 16.597826043272015, 'Var': 105.43592779285161, 'ASN': 7922, 'Mean': 16.597826043272015}
ASN:7922 1013 2/19
atlasRequest
atlas success
{'Std': 89.19682150409469, 'MSM_ID': 1013, 'Timestamp': 1582070400, 'Median': 15.6917796667, 'RTT': 20.23814170127213, 'Var': 7956.072966433329, 'ASN': 7922, 'Mean': 20.23814170127213}
ASN:7922 1013 2/20
atlasRequest
atlas success
{'Std': 1.8813589307794694, 'MSM_ID': 1013, 'Timestamp': 1582070400, 'Median': 18.68725, 'RTT': 18.972594310188054, 'Var': 3.539511426423668, 'ASN': 30036, 'Mean': 18.972594310188054}
ASN:30036 1013 2/20
atlasRequest
atlas success
{'Std': 15.871926412015123, 'MSM_ID': 1013, 'Timestamp': 1582156800, 'Median': 15.8731555, 'RTT': 17.247959879668507, 'Var': 251.91804802842324, 'ASN': 7922, 'Mean': 17.247959879668507}
ASN:7922 1013 2/21
atlasRequest
atlas success
{'Std': 2.021294053707227, '

In [ ]:
import csv

results_file = 'atlasdata_6_7922.csv'

with open(results_file) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    timestamps = []
    rtt = []
    print csv_reader.fieldnames
    for row in csv_reader:
        print row['Timestamp'],row['MSM_ID'], row['RTT']
        if line_count == 0:
            # print('Column names are: ", join(row)})
            line_count += 1
        else:
            # print( str(row["ASN"]) + " " +  str(row["MSM_ID"]) + ' '
            #         +  str(row["Timestamp"]) + ' ' + str(row["RTT"]))
            timestamps.append((row['Timestamp']))
            rtt.append(row['RTT'])
            line_count += 1
    print('Processed lines:' + str(line_count))

print timestamps
print rtt